In [40]:
import os
import zipfile

import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [41]:
def read_data_ml100k():
    data_dir = "ml-100k"
    names = ['user_id', 'item_id', 'rating', 'timestamp']
    data = pd.read_csv(os.path.join(data_dir, 'u.data'), sep='\t',
                       names=names, engine='python')
    num_users = data.user_id.unique().shape[0]
    num_items = data.item_id.unique().shape[0]
    return data, num_users, num_items

In [42]:
def split_data_ml100k(data, num_users, num_items,
                      split_mode='random', test_ratio=0.1):
    if split_mode == 'seq-aware':
        train_items, test_items, train_list = {}, {}, []
        for line in data.itertuples():
            u, i, rating, time = line[1], line[2], line[3], line[4]
            train_items.setdefault(u, []).append((u, i, rating, time))
            if u not in test_items or test_items[u][-1] < time:
                test_items[u] = (i, rating, time)
        for u in range(1, num_users+1):
            train_list.extend(sorted(train_items[u], key=lambda k: k[3]))
        test_data = [(key, *value) for key, value in test_items.items()]
        train_data = [item for item in train_list if item not in test_data]
        train_data = pd.DataFrame(train_data)
        test_data = pd.DataFrame(test_data)
    else:
        mask = [True if x == 1 else False for x in np.random.uniform(
            0, 1, (len(data))) < 1 - test_ratio]
        neg_mask = [not x for x in mask]
        train_data, test_data = data[mask], data[neg_mask]
    return train_data, test_data

In [43]:
def load_data_ml100k(data, num_users, num_items, feedback='explicit'):
    users, items, scores = [], [], []
    inter = np.zeros((num_items, num_users)) if feedback == 'explicit' else {}
    for line in data.itertuples():
        user_index, item_index = int(line[1] - 1), int(line[2] - 1)
        score = int(line[3]) if feedback == 'explicit' else 1
        users.append(user_index)
        items.append(item_index)
        scores.append(score)
        if feedback == 'implicit':
            inter.setdefault(user_index, []).append(item_index)
        else:
            inter[item_index, user_index] = score
    return users, items, scores, inter

In [44]:
class ArrayDataset(Dataset):
    def __init__(self, u, i, r):
        self.u = u
        self.i = i
        self.r = r
    
    def __len__(self):
        return len(self.u)
    
    def __getitem__(self, idx):
        return (self.u[idx], self.i[idx], self.r[idx])

In [45]:
def split_and_load_ml100k(split_mode='seq-aware', feedback='explicit',
                          test_ratio=0.1, batch_size=256):
    data, num_users, num_items = read_data_ml100k()
    train_data, test_data = split_data_ml100k(
        data, num_users, num_items, split_mode, test_ratio)
    train_u, train_i, train_r, _ = load_data_ml100k(
        train_data, num_users, num_items, feedback)
    test_u, test_i, test_r, _ = load_data_ml100k(
        test_data, num_users, num_items, feedback)
    train_set = ArrayDataset(
        np.array(train_u), np.array(train_i), np.array(train_r))
    test_set = ArrayDataset(
        np.array(test_u), np.array(test_i), np.array(test_r))
    train_iter = DataLoader(
        train_set, shuffle=True, batch_size=batch_size)
    test_iter = DataLoader(
        test_set, batch_size=batch_size)
    return num_users, num_items, train_iter, test_iter

In [46]:
class MF(nn.Module):
    def __init__(self, num_factors, num_users, num_items):
        super().__init__()
        self.P = nn.Embedding(num_users, num_factors)
        self.Q = nn.Embedding(num_items, num_factors)
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_bias = nn.Embedding(num_items, 1)
    
    def forward(self, user_id, item_id):
        P_u = self.P(user_id)
        Q_i = self.Q(item_id)
        b_u = self.user_bias(user_id)
        b_i = self.item_bias(item_id)
        outputs = (P_u * Q_i).sum(axis=1) + np.squeeze(b_u) + np.squeeze(b_i)
        return outputs.flatten()

In [47]:
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, truth, prediction):
        criterion = nn.MSELoss()
        eps = 1e-6
        loss = torch.sqrt(criterion(truth, prediction) + eps)
        return loss

In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_factors = 8
lr = 0.001
batch_size = 512
wd = 0.0001

num_epochs = 10

In [49]:
num_users, num_items, train_iter, test_iter = split_and_load_ml100k()

In [50]:
model = MF(num_factors, num_users, num_items).to(device)
loss_fn = RMSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

In [58]:
for epoch in range(num_epochs):
    train_loss = []
    model.train()
    
    for i, values in enumerate(train_iter):
        optimizer.zero_grad()
        
        user = values[0].long().to(device)
        item = values[1].long().to(device)
        rating = values[2].float().to(device)
        
        preds = model(user, item)
        loss = loss_fn(rating, preds)
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.detach().cpu().item())
        
    test_loss = []
    model.eval()
    with torch.no_grad():
        for i, values in enumerate(test_iter):
            user = values[0].long().to(device)
            item = values[1].long().to(device)
            rating = values[2].float().to(device)
            
            preds = model(user, item)
            loss = loss_fn(rating, preds)
            test_loss.append(loss.detach().cpu().item())


    print(f"{epoch} Epochs")
    print(f"train_loss: {np.mean(train_loss):.3f}")
    print(f"test_loss: {np.mean(test_loss):.3f}")
    print("\n")

0 Epochs
train_loss: 0.886
test_loss: 1.111


1 Epochs
train_loss: 0.882
test_loss: 1.106


2 Epochs
train_loss: 0.878
test_loss: 1.102


3 Epochs
train_loss: 0.875
test_loss: 1.098


4 Epochs
train_loss: 0.872
test_loss: 1.095


5 Epochs
train_loss: 0.869
test_loss: 1.092


6 Epochs
train_loss: 0.866
test_loss: 1.089


7 Epochs
train_loss: 0.864
test_loss: 1.086


8 Epochs
train_loss: 0.861
test_loss: 1.085


9 Epochs
train_loss: 0.859
test_loss: 1.082


